#### Câu 1.2: NPS Score Analysis
- Phân tích phân bố NPS score theo từng phone_model
- Tính điểm trung bình, median và số lượng đánh giá
- Xác định **top 3 models có điểm trung bình thấp nhất**

In [0]:
select 
  case
    when phone_model like 'iPhone%' then 'iPhone'
    when phone_model like 'SM%' then 'Samsung'
    when phone_model like 'CPH%' then 'Oppo'
    else 'Other'
  end as phone_brand, 
  percentile_approx(nps_score, 0.25) AS Q1,
  median(nps_score) AS score_median,
  percentile_approx(nps_score, 0.75) AS Q3,
  avg(nps_score) AS score_avg,
  mode(nps_score) AS score_mode,
  stddev(nps_score) AS score_std,
  count(1) AS count_nps_cmt,
  count(distinct phone_model) as count_phone_model
from workspace.bi.nps_survey
where nps_score is not null
group by 1

phone_brand,Q1,score_median,Q3,score_avg,score_mode,score_std,count_nps_cmt,count_phone_model
iPhone,8,10.0,10,8.892412231030578,10,1.9245776290827807,1766,40
Oppo,8,10.0,10,8.870967741935484,10,2.0451884262641253,31,25
Samsung,8,10.0,10,8.782608695652174,10,2.187615094498598,115,63
Other,8,10.0,10,8.523809523809524,10,2.618176457628801,84,68


# Tổng quan
- Nhìn chung, phân bố điểm NPS của tất cả các nhóm thương hiệu điện thoại đều có dạng lệch trái mạnh khi Median = 10 và, có ít nhất 50% khách hàng ở mọi thương hiệu đã chấm điểm tuyệt đối (10 điểm).
- Khoảng tứ phân vị (IQR = Q3 - Q1) của tất cả các nhóm đều bằng 2 (từ điểm 8 đến 10). Điều này cho thấy sự tập trung dữ liệu rất cao ở phân khúc điểm cao. Hiện tượng "Đuôi dài": Mặc dù đa số tập trung ở điểm 10, nhưng giá trị trung bình (Mean) của tất cả các nhóm đều thấp hơn Median ( từ 8.52 đến 8.89). Điều này chứng tỏ sự tồn tại của một "cái đuôi" dài về phía bên trái - các điểm số thấp từ 0-6 đã kéo giá trị trung bình xuống.

# Phân tích chi tiết theo từng thương hiệu
- iPhone: Phân bố "Vững chãi" nhất: iPhone không chỉ có số lượng phản hồi áp đảo (1.766) mà còn có phân bố điểm ổn định nhất. Độ lệch chuẩn (Std = 1.92) thấp nhất trong các nhóm cho thấy các điểm số của iPhone ít bị phân tán hơn. "Cái đuôi" bên trái của iPhone ngắn hoặc ít điểm cực thấp hơn so với các đối thủ, giúp nó duy trì mức điểm trung bình cao nhất (8.89).
- Samsung & Oppo: Phân bố tương đồng:Hai nhóm này có đặc điểm phân bố khá giống iPhone về các mốc phân vị (Q1, Median, Q3).Tuy nhiên, Samsung (Std = 2.19) có độ phân tán rộng hơn iPhone một chút, cho thấy sự xuất hiện của các khách hàng không hài lòng (Detractors) có trọng số nặng hơn, kéo điểm trung bình xuống còn 8.78.
- Other: Phân bố "Phân hóa" mạnh nhất:Nhóm này có Độ lệch chuẩn cao nhất (2.62) và Điểm trung bình thấp nhất (8.52). Ý nghĩa: Mặc dù vẫn có ít nhất một nửa người dùng chấm điểm 10, nhưng nhóm "Other" chứa đựng nhiều "biến số" hơn. Những khách hàng dùng các dòng máy khác này có xu hướng cực đoan hơn trong trải nghiệm — hoặc là rất thích (10 điểm), hoặc là rất không hài lòng (điểm rất thấp).

In [0]:
select  
  phone_model,
  avg(nps_score) AS score_avg,
  dense_rank() over (order by avg(nps_score) asc) as rank
from workspace.bi.nps_survey
where nps_score is not null
group by 1
qualify rank <=3

phone_model,score_avg,rank
Z2352N,0.0,1
Z2458,0.0,1
Mi 9 SE,1.0,2
RMX2151,3.0,3


Kết quả top 3 Model có điểm trung bình thấp nhất: Z2352N, Z2458 và Mi 9 SE